# Discover duplicate content across documents 

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter


### Read the documents

### Chunk the documents with metadata 

### Vector embedding of the documents with metadata

### Documents duplication search
